In [1]:
import random
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt
import cloudpickle as cpkl
from pathlib import Path
import numpy as np
import datetime
from datetime import timedelta
from functools import reduce
from fbprophet import Prophet
import multiprocessing as mp
from itertools import repeat
from joblib import Parallel, delayed
import calendar
import operator

In [2]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler


In [3]:
cat_var_dict = {'store_nbr': 10, 'item_nbr': 50, 'Year': 2, 'Month': 6,
'Day': 10, 'Week': 2,'holiday': 3, 'onpromotion_x': 3, 'salary_day':3,
'city': 6, 'state': 5, 'type_x': 3, 'cluster': 5,'type_y':3,'locale':5,  
'family': 10, 'class': 20, 'perishable' :3,
 }

In [4]:
contin_vars = ['onpromotion_bw', 'onpromotion_fw','holiday_bw', 'holiday_fw','dcoilwtico','transactions',
               'Afterholiday', 'Beforeholiday','Afteronpromotion', 'Beforeonpromotion']
cat_vars = [o[0] for o in 
            sorted(cat_var_dict.items(), key=operator.itemgetter(1), reverse=True)]

In [5]:
datatypes = {'id':np.uint32,'date':str,'store_nbr':np.uint8,'item_nbr':np.uint32,
            'unit_sales':np.float32,'onpromotion_x':np.uint8,'salary_day':np.uint8,
            'holiday':np.uint8,'Afterholiday':np.int16,'Beforeholiday':np.int16,
            'holiday_bw':np.int8,'holiday_fw':np.int8,'Afteronpromotion':np.int16,
            'Beforeonpromotion':np.int16,'onpromotion_bw':np.int8,'onpromotion_fw':np.int8,
            'city':str,'state':str,'type_x':str,'cluster':np.uint8,'family':str,
            'class':np.uint16,'perishable':np.uint8,'type_y':str,'locale':str,
            'dcoilwtico':np.float16,'transactions':np.float32,'Year':np.uint16,
            'Month':np.uint8,'Week':np.uint8,'Day':np.uint8}
date_cols = ['date']

In [6]:
joined = pd.read_csv("results/joined.csv",index_col=0,usecols=datatypes.keys(),dtype=datatypes,parse_dates=date_cols)

/home/suresh/.local/lib/python3.5/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [15]:
#te = pd.read_csv("results/joined.csv",nrows=100,index_col=0)
print(joined.columns)
print(joined[joined.dcoilwtico.isnull()].date.unique())

Index(['date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion_x',
       'salary_day', 'holiday', 'Afterholiday', 'Beforeholiday', 'holiday_bw',
       'holiday_fw', 'Afteronpromotion', 'Beforeonpromotion', 'onpromotion_bw',
       'onpromotion_fw', 'city', 'state', 'type_x', 'cluster', 'family',
       'class', 'perishable', 'type_y', 'locale', 'dcoilwtico', 'transactions',
       'Year', 'Month', 'Week', 'Day'],
      dtype='object')
['2015-01-03T00:00:00.000000000' '2015-01-04T00:00:00.000000000'
 '2015-01-10T00:00:00.000000000' '2015-01-11T00:00:00.000000000'
 '2015-01-17T00:00:00.000000000' '2015-01-18T00:00:00.000000000'
 '2015-01-24T00:00:00.000000000' '2015-01-25T00:00:00.000000000'
 '2015-01-31T00:00:00.000000000' '2015-02-01T00:00:00.000000000'
 '2015-02-07T00:00:00.000000000' '2015-02-08T00:00:00.000000000'
 '2015-02-14T00:00:00.000000000' '2015-02-15T00:00:00.000000000'
 '2015-02-21T00:00:00.000000000' '2015-02-22T00:00:00.000000000'
 '2015-02-28T00:00:00.000000000' '2

In [ ]:
for v in contin_vars: joined.ix[joined[v].isnull(), v] = 0
for v in cat_vars: joined.ix[joined[v].isnull(), v] = ""

In [ ]:
cat_maps = [(o, LabelEncoder()) for o in cat_vars]
contin_maps = [([o], StandardScaler()) for o in contin_vars]

In [ ]:
cat_mapper = DataFrameMapper(cat_maps)
cat_map_fit = cat_mapper.fit(joined)
cat_cols = len(cat_map_fit.features)
cat_cols

In [ ]:
len(cat_vars)

In [ ]:
contin_mapper = DataFrameMapper(contin_maps)
contin_map_fit = contin_mapper.fit(joined)
contin_cols = len(contin_map_fit.features)
contin_cols

In [ ]:
import pickle
pickle.dump(contin_map_fit, open('results/contin_maps.pickle', 'wb'))
pickle.dump(cat_map_fit, open('results/cat_maps.pickle', 'wb'))

In [ ]:
joined.set_index("date",inplace=True)


In [ ]:
train_ratio = 0.9
train_size = int(len(joined) * train_ratio)

In [ ]:
joined_valid = joined[train_size:]
joined_train = joined[:train_size]
len(joined_valid), len(joined_train)

In [ ]:
def cat_preproc(dat):
    return cat_map_fit.transform(dat).astype(np.int32)
cat_map_train = cat_preproc(joined_train)


In [ ]:
cat_map_valid = cat_preproc(joined_valid)

In [ ]:
def contin_preproc(dat):
    return contin_map_fit.transform(dat).astype(np.float32)

In [ ]:
contin_map_train = contin_preproc(joined_train)
contin_map_valid = contin_preproc(joined_valid)

In [ ]:
y_train_orig = joined_train.unit_sales
y_valid_orig = joined_valid.unit_sales

In [ ]:
max_log_y = np.max(np.log1p(joined.unit_sales))
y_train = np.log1p(y_train_orig)/max_log_y
y_valid = np.log1p(y_valid_orig)/max_log_y
#y_train = np.log(y_train)
#ymean=y_train_orig.mean()
#ystd=y_train_orig.std()
#y_train = (y_train_orig-ymean)/ystd
#y_valid = np.log(y_valid)
#y_valid = (y_valid_orig-ymean)/ystd

In [ ]:
def rmspe(y_pred, targ = y_valid_orig):
    pct_var = (targ - y_pred)/targ
    return math.sqrt(np.square(pct_var).mean())
def NWRMSLE(y, pred, wts,lt=False):
    y = y.clip(0, y.max())
    pred = pred.clip(0, pred.max())
    #wts = np.full_like(y,wts)
    #score = np.nansum(wts * ((np.log1p(pred) - np.log1p(y)) ** 2)) / wts.sum()
    if lt:
        score = np.nansum(wts * (np.subtract((pred), (y)) ** 2)) / wts.sum()
    else:
        score = np.nansum(wts * (np.subtract(np.log1p(pred), np.log1p(y)) ** 2)) / wts.sum()
    return np.sqrt(score)
def log_max_inv(preds, mx = max_log_y):
    return np.exp(preds * mx)
def normalize_inv(preds):
    return preds * ystd + ymean

In [ ]:
#print (ymean)

In [ ]:
def split_cols(arr): return np.hsplit(arr,arr.shape[1])

In [ ]:
#map_train = split_cols(cat_map_train) + [contin_map_train]
#map_valid = split_cols(cat_map_valid) + [contin_map_valid]
split_contins = True
if split_contins:
    map_train = split_cols(cat_map_train) + split_cols(contin_map_train)
    map_valid = split_cols(cat_map_valid) + split_cols(contin_map_valid)
else:
    map_train = split_cols(cat_map_train) + [contin_map_train]
    map_valid = split_cols(cat_map_valid) + [contin_map_valid]

In [ ]:
#map_train = split_cols(cat_map_train) + split_cols(contin_map_train)
#map_valid = split_cols(cat_map_valid) + split_cols(contin_map_valid)

In [ ]:
def cat_map_info(feat): return feat[0], len(feat[1].classes_)

In [ ]:
cat_map_info(cat_map_fit.features[1])

In [ ]:
def my_init(scale):
    return lambda shape, name=None: initializers.uniform(shape, scale=scale)

In [ ]:
def emb_init(shape, name=None): 
    return initializers.uniform(shape)

In [ ]:
from keras.layers import *
def get_emb(feat):
    name, c = cat_map_info(feat)
    #c2 = cat_var_dict[name]
    c2 = (c+1)//2
    if c2>100: c2=100
    inp = Input((1,), dtype='int64', name=name+'_in')
    # , W_regularizer=l2(1e-6)
    # Embedding(vocabulary_size,output_size,input_length)
    u = Flatten(name=name+'_flt')(Embedding(c, c2, input_length=1, embeddings_initializer='uniform')(inp))
#     u = Flatten(name=name+'_flt')(Embedding(c, c2, input_length=1)(inp))
    return inp,u

In [ ]:
def get_contin(feat):
    name = feat[0][0]
    inp = Input((1,), name=name+'_in')
    return inp, Dense(1, name=name+'_d')(inp)

In [ ]:
if split_contins:
    conts = [get_contin(feat) for feat in contin_map_fit.features]
    cont_out = [d for inp,d in conts]
    cont_inp = [inp for inp,d in conts]
else:
    contin_inp, contin_out = get_contin_one()
    cont_out = [contin_out]
    cont_inp = [contin_inp]
#contin_inp = Input((contin_cols,), name='contin')
#contin_out = Dense(contin_cols*10, activation='relu', name='contin_d')(contin_inp)

#contin_out = BatchNormalization()(contin_out)

In [ ]:
from keras.models import Model
embs = [get_emb(feat) for feat in cat_map_fit.features]
#conts = [get_contin(feat) for feat in contin_map_fit.features]
#contin_d = [d for inp,d in conts]
x = merge([emb for inp,emb in embs] + cont_out, mode='concat')
#x = merge([emb for inp,emb in embs] + contin_d, mode='concat')

x = Dropout(0.02)(x)
x = Dense(1000, activation='relu', init='uniform')(x)
x = Dense(500, activation='relu', init='uniform')(x)
x = Dropout(0.2)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model([inp for inp,emb in embs] + cont_inp, x)
#model = Model([inp for inp,emb in embs] + [inp for inp,d in conts], x)
model.compile('adam', 'mean_squared_error')
#model.compile(Adam(), 'mse')

In [ ]:
print(cat_map_train.shape)
print(len(split_cols(cat_map_train)))

In [ ]:
%%time 
hist = model.fit(map_train, y_train, batch_size=512, nb_epoch=20,
                 verbose=2, validation_data=(map_valid, y_valid))

In [ ]:
cat_map_train.shape

In [ ]:
hist1 = model.fit(map_train, y_train, batch_size=512, nb_epoch=20,
                 verbose=2, validation_data=(map_valid, y_valid))

In [ ]:
contin_map_train.shape

In [ ]:
len(split_cols(cat_map_train))

In [ ]:
len(split_cols(cat_map_train) + [contin_map_train])

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.show()

In [ ]:
preds = np.squeeze(model.predict(map_valid, 1024))

In [ ]:
np.percentile(preds,np.arange(0,100,10))


In [ ]:
preds[:10]

In [ ]:
cat_map_train[:10]

In [ ]:
print(contin_vars)
print(contin_map_train[:10])

In [ ]:
joined.dcoilwtico.unique()